In [ ]:
import requests


url = "http://api.weatherstack.com/historical"
params = {
    "access_key": "df6f62c7bff5489ea030bcd96b725280",
    "query": "51.5074,-0.1278",  # Latitude, Longitude pour Londres
    "historical_date": "2024-01-01"
}
response = requests.get(url, params=params)
print(response.json())

